In [1]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=9NIAETM-DJcSDrfw4Po4YRH1Cx_u5iEocPMIwpRVU1M&tc=u95MJ1IiinOCBqK0lSslvcd23TAcGlp73c0qV3IyKgw&cc=BO3Gedm8W4i3Bv63OMZBMltkLU6Ys5xqANnLwygERiI

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AWgavdfVZD6kJAMqLw5nB4uCvfdpdQbvPjfl7YGdEBA-fdv2e7rJpFgnDfM

Successfully saved authorization token.


In [2]:
def select(long1, lat1, long2, lat2):
    rectangle = ee.Geometry.Rectangle([long1, lat1, long2, lat2])

    dataset = 'LANDSAT/LC08/C01/T1_TOA'

    l8 = ee.ImageCollection(dataset)

    img = ee.Image(
        l8.filterBounds(rectangle)
            .filterDate('2015-01-01',
                '2015-12-31')
            .sort('CLOUD_COVER').first()
    )
    return img.clip(rectangle).select(list(range(11)))

In [3]:
import pandas as pd
latdata = pd.read_csv('/content/drive/MyDrive/SOCData/latdata.csv').to_numpy()
latdata.shape

(22351710, 5)

In [4]:
def SFIM(img, pan):
    imgScale = img.projection().nominalScale()
    panScale = pan.projection().nominalScale()

    kernelWidth = imgScale.divide(panScale)
    kernel = ee.Kernel.square(radius = kernelWidth.divide(2))

    panSmooth = pan.reduceNeighborhood(
    reducer = ee.Reducer.mean(),
    kernel = kernel
    )

    img = img.resample("bicubic")
    sharp = img.multiply(pan).divide(panSmooth).reproject(pan.projection())
    return sharp

In [101]:
def pan_sharpen (img, bands):
    # print(type(img))
    to_sharpen = img.select(bands);
    # Select the 15 m panchromatic band
    pan = img.select(["B8"]);

    return SFIM(to_sharpen, pan).addBands(pan)

In [6]:
!pip install spyndex

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 725.6/725.6 KB 19.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.6/123.6 KB 16.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 81.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.1/198.1 KB 23.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for spyndex: filename=spyndex-0.2.0-py3-none-any.whl size=765264 sha256=029667daef66f3e072912bf709f7617f6a784e069bceded19a430abbebc9c40e
  Stored in directory: /root/.cache/pip/wheels/f2/27/4e/fd9eedfc506ed7ce975502ad811e9ff836cd2b8175d89b802a
  Created wheel for eemont: filename=eemont-0.3.5-py3-none-any.whl size=123958 sha256=d60ae302ce84be320608a274f967e4ad3b350bd4da30ab61afb902aaab1a30ab
  Stored in directory: /root/.cache/pip/wheel

In [83]:
list(latdata[0])

[31750.0, -16.7833333333986, -16.7750000000653, 13.9916666666251, 14.0]

In [7]:
import spyndex

In [102]:
def compute_indices (img):
    dataset = 'LANDSAT/LC08/C01/T1_TOA'

    parameters = {
        "A": img.select("B1"),
        "B": img.select("B2"),
        "G": img.select("B3"),
        "R": img.select("B4"),
        "N": img.select("B5"),
        "S1": img.select("B6"),
        "S2": img.select("B7"),
        "T1": img.select("B10"),
        "T2": img.select("B11"),
        "L" : 1, 
        "g" : 2.5, "C1" : 6, "C2" : 7.5
    }

    indices = ['AFRI1600','AFRI2100','ANDWI','AVI','AWEInsh','AWEIsh','BAI','BAIM','BCC','BI','BLFEI','BNDVI','BRBA','BaI','CIG','CSI','CSIT','CVI','DBI','DBSI','DSI','DSWI1','DSWI2','DSWI3','DSWI4','DSWI5','DVI','EBBI','EMBI','EVI','EVI2','ExG','ExGR','ExR','FCVI','GARI','GBNDVI','GCC','GEMI','GLI','GNDVI','GOSAVI','GRNDVI','GRVI','GSAVI','GVMI','IBI','IKAW','IPVI','LSWI','MBI','MCARI1','MCARI2','MGRVI','MIRBI','MLSWI26','MLSWI27','MNDVI','MNDWI','MNLI','MRBVI','MSAVI','MSI','MSR','MTVI1','MTVI2','MuWIR','NBAI','NBLI','NBR','NBR2','NBRSWIR','NBRT1','NBRT2','NBRT3','NBSIMS','NBUI','NDBI','NDBaI','NDDI','NDGlaI','NDII','NDISIb','NDISIg','NDISImndwi','NDISIndwi','NDISIr','NDMI','NDPonI','NDSI','NDSII','NDSWIR','NDSaII','NDTI','NDVI','NDVIMNDWI','NDVIT','NDWI','NDYI','NGRDI','NIRv','NLI','NMDI','NRFIg','NRFIr','NSDS','NSDSI1','NSDSI2','NSDSI3','NSTv1','NSTv2','NWI','NormG','NormNIR','NormR','OSAVI','PISI','RCC','RDVI','RGBVI','RGRI','RI','RI4XS','S3','SARVI','SAVI','SAVIT','SI','SIPI','SR','SR2','SWI','SWM','TDVI','TGI','TVI','TriVI','UI','VARI','VI6T','VIBI','VIG','VgNIRBI','VrNIRBI','WI1','WI2','WI2015','WRI']
    

    for x in range(0, len(indices), 2):
        try:
            img = img.addBands(spyndex.computeIndex([indices[x], indices[x + 1]], parameters))
        except Exception as e:
            pass

    meanDict = img.reduceRegion(
        reducer= ee.Reducer.mean(),
        geometry= img.geometry(),
        scale= 90,
        maxPixels= 40e9
    )

    # mappedDictionary = meanDict.map(lambda key, val:  val if val is not None else 0)

    return meanDict


In [109]:
import numpy as np
sample = latdata[np.random.randint(latdata.shape[0], size=10000), :]

In [ ]:
def process_image(long1, lat1, long2, lat2, MU_GLOBAL):
	# Combine selecting area, pan-sharpening, illumination, topographic correction, and spectral indices

    img = select(long1, lat1, long2, lat2)
    img = img.scaleAndOffset()

    img = pan_sharpen(img, ["B1", "B2", "B3", "B4", "B5", "B6", "B7","B9", "B10", "B11"])
    indices = compute_indices(img);

    return indices.set('MU_GLOBAL', MU_GLOBAL).getInfo()

features = [];



for i in range(len(sample)):
    processed = process_image(sample[i][1], sample[i][3], sample[i][2], sample[i][4], sample[i][0]);
    features.append(processed);
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64


In [ ]:
df =  pd.DataFrame(features)

In [ ]:
df.to_csv("/content/drive/MyDrive/SOC/Data/TOAFastFeatures")